In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
sys.path.append('../..')
from modules.many_features import utils, constants, env
from modules.many_features.random_agent import RandomAgent
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [3]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df = pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df = pd.read_csv('../../data/more_features/more_feats_correlated_noisy_8.csv')
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.317680,-1.000000,2.088832,-1.000000,-1.000000,90.986272,113.064122,3.731666,1,1.276701,81.145964,78.146183,29.513433,26.029977,105.808765,33.953040,-1.000000,Aplastic anemia
1,10.109186,-1.000000,-1.000000,-1.000000,-1.000000,104.941225,107.239455,2.889957,0,1.191488,86.444945,103.728730,32.453257,20.009164,90.105559,30.327558,-1.000000,Unspecified anemia
2,6.289434,-1.000000,4.156000,-1.000000,-1.000000,86.013564,-1.000000,2.193642,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,18.868301,-1.000000,Hemolytic anemia
3,7.077683,97.195746,0.261727,0.142422,263.478460,94.058079,102.335651,2.257440,1,-1.000000,114.702341,64.345736,20.102258,3.865156,-1.000000,21.233048,38.840234,No anemia
4,9.717874,101.889169,-1.000000,3.937127,475.993105,78.258930,-1.000000,3.725277,0,1.367858,-1.000000,-1.000000,38.308559,-1.000000,-1.000000,29.153622,-1.000000,No anemia


In [4]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

#### The Random Agent

In [5]:
%%time
rand_agent = RandomAgent(X_test[0:], y_test[0:])
test_df = rand_agent.test()
test_df.head()

Testing done.....
Wall time: 3min 22s


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,2.0,0.0,0.0,-1.0,0.0,"[ret_count, Vitamin B12/Folate deficiency anemia]",4.0,1.0
1,1.0,1.0,0.0,-1.0,0.0,[No anemia],4.0,0.0
2,2.0,2.0,0.0,-1.0,0.0,"[segmented_neutrophils, Iron deficiency anemia]",5.0,4.0
3,1.0,3.0,0.0,-1.0,0.0,[Anemia of chronic disease],7.0,3.0
4,1.0,4.0,0.0,-1.0,0.0,[Anemia of chronic disease],5.0,3.0


In [ ]:
# %%time
# rand_agent.test_sample(2)

In [ ]:
# from datetime import datetime
# start = datetime.now()
# rand_agent.test_sample(2)
# end = datetime.now()
# duration = end-start
# duration

In [ ]:
len(X_test), len(test_df)

In [ ]:
# test_df.to_csv('../../test_dfs/many_features/random_test_df_noisy_0.4.csv', index=False)

In [6]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

11.857142857142858

In [7]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(1.5399285714285715, -0.768)

In [8]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.11857142857142858, 0.11740218948461312, 0.49864539085761783)

In [9]:
test_df.y_pred.unique()

array([1., 0., 4., 3., 2., 6., 7., 5.])